# Education Data Analysis with Database Integration

This notebook presents a comprehensive analysis of education data across different countries, with data stored and analyzed using both PostgreSQL and MongoDB databases.

## Project Components
1. Database Setup and Configuration
2. Data Collection and Storage
3. Data Analysis
4. Time Series Forecasting
5. Visualization

## Prerequisites
- PostgreSQL database
- MongoDB database
- Required Python packages

First, ensure that both PostgreSQL and MongoDB are running using Docker:

In [ ]:
# Check if Docker containers are running
!docker ps

## 1. Environment Setup

Install required packages and import necessary libraries

In [ ]:
# Install required packages
!pip install pandas numpy plotly scikit-learn statsmodels pymongo psycopg2-binary python-dotenv eurostat tqdm

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
import eurostat
import logging
from datetime import datetime
import os
from dotenv import load_dotenv
from pymongo import MongoClient
import psycopg2
from psycopg2.extras import execute_values

# Configure logging
logging.basicConfig(
    filename='education_data_collection.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

## 2. Database Configuration

Set up connections to PostgreSQL and MongoDB

In [ ]:
# Load environment variables
load_dotenv()

# PostgreSQL configuration
POSTGRES_CONFIG = {
    'dbname': os.getenv('POSTGRES_DB', 'education_db'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'postgrespassword'),
    'host': os.getenv('POSTGRES_HOST', 'localhost'),
    'port': os.getenv('POSTGRES_PORT', '5432')
}

# MongoDB configuration
MONGODB_CONFIG = {
    'host': os.getenv('MONGO_HOST', 'localhost'),
    'port': int(os.getenv('MONGO_PORT', '27017')),
    'db': os.getenv('MONGO_DB', 'education_db')
}

# Test database connections
try:
    # PostgreSQL connection
    pg_conn = psycopg2.connect(**POSTGRES_CONFIG)
    print("Successfully connected to PostgreSQL")
    
    # MongoDB connection
    mongo_client = MongoClient(MONGODB_CONFIG['host'], MONGODB_CONFIG['port'])
    mongo_db = mongo_client[MONGODB_CONFIG['db']]
    print("Successfully connected to MongoDB")
    
except Exception as e:
    print(f"Error connecting to databases: {str(e)}")

## 3. Database Schema Setup

Create necessary tables in PostgreSQL

In [ ]:
def setup_postgres_schema(conn):
    """Set up PostgreSQL database schema"""
    try:
        with conn.cursor() as cur:
            # Create tables for raw data
            cur.execute("""
                CREATE TABLE IF NOT EXISTS raw_education_data (
                    id SERIAL PRIMARY KEY,
                    country VARCHAR(50),
                    year INTEGER,
                    metric_name VARCHAR(50),
                    metric_value FLOAT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );

                CREATE TABLE IF NOT EXISTS analysis_results (
                    id SERIAL PRIMARY KEY,
                    analysis_type VARCHAR(50),
                    metric_name VARCHAR(50),
                    year INTEGER,
                    country VARCHAR(50),
                    result_value FLOAT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );

                CREATE TABLE IF NOT EXISTS forecasts (
                    id SERIAL PRIMARY KEY,
                    metric_name VARCHAR(50),
                    forecast_year INTEGER,
                    forecast_value FLOAT,
                    confidence_lower FLOAT,
                    confidence_upper FLOAT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                );
            """)
            conn.commit()
            print("PostgreSQL schema setup completed")
    except Exception as e:
        print(f"Error setting up PostgreSQL schema: {str(e)}")

# Set up schema
setup_postgres_schema(pg_conn)

## 4. Data Collection and Storage

Collect education data from Eurostat and store in databases

In [ ]:
def collect_and_store_education_data(pg_conn, mongo_db):
    """Collect education data from Eurostat and store in databases"""
    datasets = {
        'education_investment': 'educ_uoe_fine09',
        'student_teacher_ratio': 'educ_uoe_perp04',
        'completion_rate': 'edat_lfse_03',
        'literacy_rate': 'edat_lfse_01'
    }
    
    for metric, code in datasets.items():
        try:
            print(f"Collecting {metric} data...")
            df = eurostat.get_data_df(code)
            
            # Process data
            if 'geo\\TIME_PERIOD' in df.columns:
                df[['country', 'year']] = df['geo\\TIME_PERIOD'].str.split('\\', expand=True)
                df = df.drop('geo\\TIME_PERIOD', axis=1)
            
            # Transform to long format
            year_columns = [col for col in df.columns if col.isdigit()]
            id_vars = [col for col in df.columns if not col.isdigit()]
            
            df_long = df.melt(
                id_vars=id_vars,
                value_vars=year_columns,
                var_name='year',
                value_name=metric
            )
            
            # Store in PostgreSQL
            with pg_conn.cursor() as cur:
                values = [(
                    row['country'],
                    int(row['year']),
                    metric,
                    float(row[metric]) if pd.notnull(row[metric]) else None
                ) for _, row in df_long.iterrows()]
                
                execute_values(cur, """
                    INSERT INTO raw_education_data (country, year, metric_name, metric_value)
                    VALUES %s
                    ON CONFLICT (country, year, metric_name) 
                    DO UPDATE SET metric_value = EXCLUDED.metric_value;
                """, values)
            pg_conn.commit()
            
            # Store in MongoDB
            collection = mongo_db[metric]
            records = df_long.to_dict('records')
            for record in records:
                collection.update_one(
                    {'country': record['country'], 'year': record['year']},
                    {'$set': record},
                    upsert=True
                )
            
            print(f"Successfully stored {metric} data")
            
        except Exception as e:
            print(f"Error processing {metric} data: {str(e)}")

# Collect and store data
collect_and_store_education_data(pg_conn, mongo_db)

## 5. Data Analysis

Analyze education metrics from the database

In [ ]:
def analyze_education_metrics(pg_conn, metric_name):
    """Analyze education metrics from PostgreSQL database"""
    try:
        with pg_conn.cursor() as cur:
            # Calculate basic statistics
            cur.execute("""
                SELECT 
                    year,
                    AVG(metric_value) as mean_value,
                    STDDEV(metric_value) as std_value,
                    MIN(metric_value) as min_value,
                    MAX(metric_value) as max_value
                FROM raw_education_data
                WHERE metric_name = %s
                GROUP BY year
                ORDER BY year;
            """, (metric_name,))
            
            results = cur.fetchall()
            stats_df = pd.DataFrame(results, 
                                  columns=['year', 'mean', 'std', 'min', 'max'])
            
            # Create visualizations
            trend_fig = px.line(stats_df, x='year', y='mean',
                              title=f'{metric_name} Trend Over Time')
            trend_fig.show()
            
            # Print statistics
            print(f"\nStatistics for {metric_name}:")
            print(stats_df)
            
            return stats_df
            
    except Exception as e:
        print(f"Error analyzing {metric_name}: {str(e)}")
        return None

# Analyze each metric
metrics = ['education_investment', 'student_teacher_ratio', 
          'completion_rate', 'literacy_rate']

for metric in metrics:
    print(f"\nAnalyzing {metric}...")
    analyze_education_metrics(pg_conn, metric)

## 6. Time Series Forecasting

Forecast future values for each metric

In [ ]:
def forecast_metric(pg_conn, metric_name, periods=5):
    """Forecast future values for a metric"""
    try:
        with pg_conn.cursor() as cur:
            # Get historical data
            cur.execute("""
                SELECT year, AVG(metric_value) as avg_value
                FROM raw_education_data
                WHERE metric_name = %s
                GROUP BY year
                ORDER BY year
            """, (metric_name,))
            
            results = cur.fetchall()
            historical_df = pd.DataFrame(results, columns=['year', 'value'])
            
            # Fit SARIMA model
            model = SARIMAX(historical_df['value'], 
                          order=(1, 1, 1), 
                          seasonal_order=(1, 1, 1, 12))
            results = model.fit()
            
            # Generate forecast
            forecast = results.forecast(periods)
            
            # Create visualization
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=historical_df['year'], 
                                   y=historical_df['value'],
                                   name='Historical'))
            fig.add_trace(go.Scatter(x=range(historical_df['year'].max() + 1, 
                                            historical_df['year'].max() + periods + 1),
                                   y=forecast,
                                   name='Forecast'))
            fig.update_layout(title=f'{metric_name} Forecast')
            fig.show()
            
            print(f"\nForecast values for {metric_name}:")
            forecast_df = pd.DataFrame({
                'year': range(historical_df['year'].max() + 1, 
                             historical_df['year'].max() + periods + 1),
                'forecast': forecast
            })
            print(forecast_df)
            
            return forecast_df
            
    except Exception as e:
        print(f"Error forecasting {metric_name}: {str(e)}")
        return None

# Generate forecasts for each metric
for metric in metrics:
    print(f"\nForecasting {metric}...")
    forecast_metric(pg_conn, metric)

## 7. Cleanup

Close database connections

In [ ]:
# Close database connections
if pg_conn:
    pg_conn.close()
    print("PostgreSQL connection closed")
    
if mongo_db:
    mongo_db.client.close()
    print("MongoDB connection closed")